In [1]:
from typing import Optional, List
from pydantic import BaseModel, Field, ValidationError
import wikipedia
import pprint
import cohere
import json

In [2]:
co = cohere.Client("3JE437qflNp5M7TbvoQXMmeDuRrRWgylGCq09x29")

In [3]:
def fetch_wikipedia_summary(instituion_name):
    instituion_page = wikipedia.summary(instituion_name, sentences = 4)
    return instituion_page

In [4]:
details_format = {
    "institution_name": "str",
    "founded_year" :"int",
   "founded_by": "str",
    "no_of_employees": "str",
    "branches": "List[str]",
    "courses": "List[str]",
}
class InstitutionDetails(BaseModel):
    institution_name: str
    founded_year: int
    founded_by: str
    no_of_employees: str
    branches: List[str]
    courses: List[str]

In [5]:
def format_wiki_summary(summary):
    response = co.generate(prompt=f"""
    Summarize the following text: ||{summary}|| in this json format {details_format} that I can easily parse using json.loads
    """)
    return response.generations[0].text.strip()

In [6]:
def get_institution_as_object()->InstitutionDetails:
    institution_name = input("Enter institution name to get summary: ")
    summary = fetch_wikipedia_summary(institution_name)
    formatted_summary = format_wiki_summary(summary)
    json_summary = formatted_summary[formatted_summary.index('{'): formatted_summary.index('}')+1]
    json_data = json.loads(json_summary)
    institution_details = InstitutionDetails(**json_data)
    return institution_details

In [7]:
def solution():
    institution = get_institution_as_object()
    print("===============================================================")
    print("Institution Name: ", institution.institution_name)
    print("Founded in: ", institution.founded_year)
    print("Founded by: ", institution.founded_by)
    print("Employee count: ", institution.no_of_employees)
    print("Departments: ", institution.branches)
    print("Courses Offered: ", institution.courses)
    print("===============================================================")

In [8]:
solution()
solution()
solution()

Enter institution name to get summary:  Bangalore Institute of Technology


Institution Name:  Bangalore Institute of Technology
Founded in:  1979
Founded by:  Rajya Vokkaligara Sangha, Bengaluru
Employee count:  Unknown
Departments:  ['Artificial Intelligence & Machine Learning', 'Computer Science', 'Electronics', 'Telecommunication', 'Instrumentation Technology', 'Electrical', 'Civil and Mechanical', 'Information Science and Engineering', 'Industrial Engineering and Management']
Courses Offered:  ['Undergraduate', 'Postgraduate', 'MBA', 'MCA', 'Ph.D.']


Enter institution name to get summary:  New Horizon College of Engineering


Institution Name:  New Horizon College of Engineering
Founded in:  2001
Founded by:  New Horizon Educational Institution
Employee count:   Undisclosed
Departments:  ['Marathahalli, Bangalore']
Courses Offered:  ['Engineering']


Enter institution name to get summary:  Christ University


Institution Name:  Christ University
Founded in:  2008
Founded by:  Carmelites of Mary Immaculate (CMI)
Employee count:  Unknown
Departments:  ['Bengaluru, Karnataka, India']
Courses Offered:  ['Undergraduate, postgraduate and doctoral programmes in arts, commerce, science, and management.']
